# Exercises:

Answer the following questions.

## 1. Import the `Requests` package and call the [GitHub Jobs](https://jobs.github.com) API with the parameters of your choice. Print the status code to make sure it went well.

In [21]:
import requests
parameter={"full_time":True}
url="http://jobs.github.com/positions.json"
r=requests.get(url,params=parameter)
r.status_code

200

## 2. Import `json` and print the whole json response with the `json.dumps()` function. See what kind of data it contains.


In [22]:
import json
r.headers
data=r.json()
print (len(data))
print(json.dumps(data, sort_keys=True,indent=2))

50
[
  {
    "company": "GitHub",
    "company_logo": "http://github-jobs.s3.amazonaws.com/e54db6c2-080d-11e7-9d25-531751d26d63.png",
    "company_url": "https://github.com/about/jobs",
    "created_at": "Wed Apr 12 16:56:04 UTC 2017",
    "description": "<p>More and more companies are using GitHub Enterprise, so the Support team is growing. When our customers have problems, they contact us and it is our goal to help them as quickly and awesomely as possible. We&#39;re looking for an Enterprise Support Manager, in Asia Pacific, who will be responsible for the oversight and performance of a distributed staff answering customer support requests, triaging issues, and providing solutions. Candidates must currently be located in Australia, Japan, or New Zealand and possess legal working authorization.</p>\n\n<p>The position reports to the Director of Enterprise Support and works closely with other teams and leaders across the company.</p>\n\n<p>GitHub works remotely and asynchronously as mu

## 3. Get the dictionary of one of the jobs and call the `keys()` method to have a better idea of the data at hand.

In [23]:
data[0].keys()

dict_keys(['created_at', 'id', 'how_to_apply', 'title', 'description', 'url', 'company', 'company_logo', 'location', 'company_url', 'type'])

## 4. Print some of the first job's content, including images and HTML. You might need to import the `IPython` package.

In [24]:
from IPython.display import Image
image_url = data[0]['company_logo']
Image(url=image_url) 

In [5]:
from IPython.core.display import display, HTML
display(HTML(data[0]['description']))

In [25]:
print (data[0]['company'],
      data[0]['company_url'],
      data[0]['how_to_apply'],
    data[0]['created_at'], sep='\n')
print(type(data[0]['created_at']))

GitHub
https://github.com/about/jobs
<p>Interested? We would love to hear more about you and your interest in joining us at GitHub! Just fill out our application <a href="https://boards.greenhouse.io/github/jobs/586192#.WMbOuxIrLBI">here</a>!</p>

Wed Apr 12 16:56:04 UTC 2017
<class 'str'>


## 5. Import `SQLite` and create a database.

In [26]:
import sqlite3
db=sqlite3.connect("database.db",detect_types=sqlite3.PARSE_DECLTYPES)
cursor=db.cursor()

## 6. Create 2 tables: `Company` and `Job`. Pay special attention on primary and foreign keys as well as unique and non-nullable columns. Make sure you store `created_at` as the equivalent of Python `datetime` and not `string`.


In [81]:
cursor.execute('CREATE TABLE IF NOT EXISTS company(id INTEGER PRIMARY KEY, company TEXT, company_url TEXT,company_logo)')
db.commit()

In [90]:
cursor.execute('CREATE TABLE IF NOT EXISTS job(id TEXT PRIMARY KEY NOT NULL,title TEXT,description TEXT,company_id INTEGER,how_to_apply TEXT, location TEXT, created_at TIMESTAMP, type TEXT,FOREIGN KEY (company_id) REFERENCES company(id))')
db.commit() 

In [80]:
#db.close()
#cursor.execute('DROP TABLE company')
#db.commit

<function Connection.commit>

In [89]:
#cursor.execute('DROP TABLE job')
#db.commit

<function Connection.commit>

## 7. Store the data you got from the API request in the database. Make sure you create a transaction (i.e., if an error occurs while saving a job's location data the database should rollback on all statements regarding that job).

In [82]:
#for i in range(len(data)):
#    comp=data[i]['company'] or ""
#    url=data[i]['company_url'] or ""
#    logo=data[i]['company_logo'] or ""
#    cursor.execute("INSERT OR IGNORE INTO company(id,company,company_url,company_logo) VALUES ("+str(i)+",'"+comp+"','"+url+"','"+logo+"')")
#db.commit()

def insert_comp(comp , url , logo):
    cursor.execute("select id from company where company='{}'".format(comp,))
    comp_id=cursor.fetchone()
    if comp_id is None:
        cursor.execute("select max(id) from company ")
        max_id=cursor.fetchone()[0]
        if max_id is None:
            cursor.execute("INSERT OR IGNORE INTO company(id,company,company_url,company_logo) VALUES (?,?,?,?)",(1 , comp, url, logo))
        else:
            new_id=max_id+1
            cursor.execute("INSERT OR IGNORE INTO company(id,company,company_url,company_logo) VALUES (?,?,?,?)",(new_id , comp, url, logo))

for row in data:
                   insert_comp(row['company'],row['company_url'],row['company_logo'])

db.commit()

In [91]:
from datetime import datetime
def into_date(date_string):
    return datetime.strptime(date_string, '%a %b %d %X %Z %Y')

#Wed Apr 12 16:56:04 UTC 2017
def insert_job(id , title, descript, comp,apply, loc, created_at, typ):
    cursor.execute("select id from company where company='{}'".format(comp,))
    comp_id=cursor.fetchone()[0]
    
    cursor.execute("INSERT OR IGNORE INTO job (id ,title,description ,company_id ,how_to_apply , location ,created_at,type) VALUES (?,?,?,?,?,?,?,?)",(id , title, descript, comp_id,apply, loc, created_at, typ))


for row in data:
    insert_job(row['id'],row['title'],row['description'],row['company'],row['how_to_apply'],row['location'],into_date(row['created_at']),row['type'])

db.commit()


## 8. Execute simple SQL queries in each table and make sure everything looks as was intended.

In [88]:
cursor.execute('SELECT id, company, company_url FROM company ')
rows=cursor.fetchall()

for row in rows:
    print(row)


(1, 'GitHub', 'https://github.com/about/jobs')
(2, 'Free2Move', 'https://free2move.com/')
(3, 'Spartez', 'https://spartez.com')
(4, 'Paf', 'http://www.paf.com')
(5, 'The Black Tux', None)
(6, 'Chegg', 'http://www.chegg.com')
(7, 'CircleCI', 'http://www.circleCI.com ')
(8, 'RBC Investor & Treasury Services', 'http://www.rbc.com/techjobs/')
(9, 'Jane Street', 'http://janestreet.com')
(10, 'MindSky', 'http://www.mindsky.com')
(11, 'Major League Hacking', 'https://mlh.io/')
(12, 'Schrödinger', 'http://www.schrodinger.com')
(13, 'Nedap', 'http://nedap.com')
(14, 'Arcesium LLC', 'http://www.arcesium.com/')
(15, 'LINE', 'http://stackoverflow.com/jobs/companies/line-corporation')
(16, 'Dekeo', None)
(17, 'Text100', 'http://www.text100.com/')
(18, 'Exact', 'http://www.exact.com/nl/nl/')
(19, 'Tripaneer', 'http://www.tripaneer.com')
(20, 'Rally Health', 'http://www.rallyhealth.com')
(21, 'Apliant', 'http://www.apliant.com')
(22, 'European Stability Mechanism (ESM)', 'https://www.esm.europa.eu/ca

In [92]:
cursor.execute('SELECT company_id, max(created_at) FROM job group by company_id')
rows=cursor.fetchall()

for row in rows:
    print(row)

(1, '2017-04-12 16:56:04')
(2, '2017-04-07 09:31:17')
(3, '2017-04-07 09:17:34')
(4, '2017-04-07 08:38:04')
(5, '2017-04-07 00:07:11')
(6, '2017-04-06 20:26:30')
(7, '2017-04-06 19:06:35')
(8, '2017-04-06 17:02:02')
(9, '2017-04-06 14:14:26')
(10, '2017-04-06 10:09:16')
(11, '2017-04-05 19:10:01')
(12, '2017-04-05 14:19:34')
(13, '2017-04-05 12:45:08')
(14, '2017-04-05 11:23:02')
(15, '2017-04-05 05:16:14')
(16, '2017-04-05 04:47:07')
(17, '2017-04-04 20:54:59')
(18, '2017-04-04 13:17:03')
(19, '2017-04-04 09:40:18')
(20, '2017-04-03 23:20:52')
(21, '2017-04-03 20:42:21')
(22, '2017-04-03 12:52:31')
(23, '2017-04-02 19:38:38')
(24, '2017-04-02 19:33:05')
(25, '2017-04-01 02:49:04')
(26, '2017-03-31 16:36:00')
(27, '2017-03-31 16:06:44')


## 9. Create queries the following queries:
 * Find out how many jobs have the phrase `Data Science` and how many have the word `Python` in the description.
 * Find out how many were created in the past 7 days.
 * Find out how many jobs are located in `California` and which companies those are with.

**The output from our queries should be similar to this:**

There are 27 jobs with the 'Data Science' term, and 40% of them have 'Python' in the description.

22% of them have been created in the past 7 days.

2 of those are in California. The companies with those jobs are: ['Carbon Black, Inc.', 'Kaggle']

In [93]:
search='Data Science'
cursor.execute("SELECT company_id,location,description FROM job where description like '%{0}%'".format(search,))
rows=cursor.fetchall()
#with PYthon
search2='Python'
cursor.execute("SELECT count(*) FROM job where description like '%{0}%' and description like '%{1}%'".format(search,search2))
rows2=cursor.fetchone()

print("There are {0} jobs with the '{1}' term, and {2}% of them have '{3}' in the description.".format(len(rows) , search , round(rows2[0]/len(rows)*100,2) , search2))
#

There are 1 jobs with the 'Data Science' term, and 100.0% of them have 'Python' in the description.


In [115]:
from datetime import timedelta

cursor.execute("SELECT company_id,created_at FROM job WHERE created_at >= ?", (datetime.today() - timedelta(days=7),))
rows_7days=cursor.fetchall()

cursor.execute("SELECT count(*) FROM job")
rows_all=cursor.fetchone()[0]
print("{}% of them have been created in the past 7 days.".format(len(rows_7days)/rows_all*100, ))

50
80.0% of them have been created in the past 7 days.


In [117]:
cursor.execute("SELECT distinct c.company FROM job j inner join company c on c.id=j.company_id where location like 'California' or location like '% CA%' ")
rows=cursor.fetchall()
company=[]
for row in rows:
    company.append(row[0])
print(len(rows), "of those are in California. The companies with those jobs are: ", company)


6 of those are in California. The companies with those jobs are:  ['GitHub', 'Chegg', 'CircleCI', 'Rally Health', 'Motive Interactive', 'Booz Allen Hamilton']
